In [1]:
stock_ = 'TATASTEEL'

In [2]:
from nsepy import get_history as gh
import datetime as dt
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [47]:
def get_regressor(stk_data, start, end, lim):
    stk_data['Date'] = stk_data.index
    data2 = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])
    data2['Date'] = stk_data['Date']
    data2['Open'] = stk_data['Open']
    data2['High'] = stk_data['High']
    data2['Low'] = stk_data['Low']
    data2['Close'] = stk_data['Close']
    
    train_set = data2.iloc[:, 1:2].values
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(train_set)
    X_train = []
    y_train = []
    for i in range(60, lim):
        X_train.append(training_set_scaled[i-60:i, 0])
        y_train.append(training_set_scaled[i, 0]) 
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    regressor.fit(X_train, y_train, epochs = 1, batch_size = 32, validation_split=0.2)
    
    return (regressor, data2, sc)

In [66]:
def comparison(start, end, lim, regressor, heading, data2, sc):
    testdataframe= gh(symbol=stock_,start=dt.datetime(2020,1,1),end=dt.datetime(2020,8,31))
    testdataframe['Date'] = testdataframe.index
    testdata = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])
    testdata['Date'] = testdataframe['Date']
    testdata['Open'] = testdataframe['Open']
    testdata['High'] = testdataframe['High']
    testdata['Low'] = testdataframe['Low']
    testdata['Close'] = testdataframe['Close']
    real_stock_price = testdata.iloc[:, 1:2].values
    dataset_total = pd.concat((data2['Open'], testdata['Open']), axis = 0)
    inputs = dataset_total[len(dataset_total) - len(testdata) - 60:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    #upper limit is number of days - 25
    for i in range(60, lim):
        X_test.append(inputs[i-60:i, 0])
    X_test = np.asarray(X_test).astype(np.float32)
    print(X_test.shape)
    X_test = np.reshape(X_test, (X_test.shape[0], -1, 1))
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    plt.figure(figsize=(20,5))
    print(heading)
    plt.plot(real_stock_price, color = 'green', label = '{} Stock Price'.format(stock_))
    plt.plot(predicted_stock_price, color = 'red', label = 'Predicted {} Stock Price'.format(stock_))
    plt.title('{} Stock Price Prediction'.format(stock_))
    plt.xlabel('Trading Day')
    plt.ylabel('{} Stock Price'.format(stock_))
    plt.legend()
    plt.show()

In [67]:
def trainer():
    start_pre = dt.datetime(2014,1,1)
    end_pre = dt.datetime(2019,6,1)
    stk_data_pre = gh(symbol=stock_,start=start_pre,end=end_pre)
    pre_lim = 1335
    reg1 = get_regressor(stk_data_pre, start_pre, end_pre, pre_lim)
    
    start_post = dt.datetime(2014,1,1)
    end_post = dt.datetime(2020,8,1)
    stk_data_post = gh(symbol=stock_,start=start_post,end=end_post)
    post_lim = 1623
    #reg2 = get_regressor(stk_data_post, start_post, end_post, post_lim)[0]
    reg2 = None
    
    #comparison(start_post,end_post, post_lim, reg2, "Corona trained on full", reg1[1])
    comparison(start_post,end_post, post_lim, reg1[0], "Corona untrained on full", reg1[1], reg1[2])
    comparison()
trainer()

32/32 [==============================] - 10s 307ms/step - loss: 0.0446 - val_loss: 0.0057


ValueError: setting an array element with a sequence.